# FalconAI - Test 1 : Implementation du modèle dynamique brut

In [3]:
import numpy as np
import os 
import tqdm

In [28]:
def Poussee(alpha, beta, thr, T):
    """Calcul de la poussée du moteur principal dans le repère de la fusée
    alpha : rotation autour de x_f
    beta : rotation autour de y_f
    thr : throttle du moteur en %
    T : poussée nominale de la fusée en N
    Wiki matrice de rotation https://fr.wikipedia.org/wiki/Matrice_de_rotation
    
    """
    
    F = thr/100*np.array((0,0,T))
    alpha = np.radians(alpha)
    beta = np.radians(beta)
    #print(F)
    
    R_x = np.array(( (1, 0, 0),
                    (0, np.cos(alpha), -np.sin(alpha)),
                    (0, np.sin(alpha),  np.cos(alpha)) ))
    
    R_y = np.array(((np.cos(beta), 0, np.sin(beta)),
                    (0, 1, 0),
                    (-np.sin(beta), 0,  np.cos(beta)) ))
    #print(R_x)
    #print(R_y)
    
    F_f = np.dot(R_y,np.dot(R_x, F))
    
    return(F_f)
    
#Poussee(3, 2, 50, 25000)

In [31]:
def RPFD(F_f, D_f, I, m):
    """
    Calcul du Principe de Fondamental de la Dynamique 
    F_f : Poussée orientée du booster dans le repère de la fusée (en N)
    D_f : Vecteur point de poussée - CDG dans le repère de la fusée (en N)
    I : Inertie du booster autour de son CDG 
    m : masse du booster (en kg)
    """
    
    x_pp = F_f / m
    
    M_f_CDG = np.cross(D_f, F_f)
    theta_pp = M_f_CDG / I
    
    return x_pp, theta_pp

In [49]:
def coord_local2global(vectors, state):
    """
    Recalcul des vecteurs dans le référentiel terrestre
    vectors : série de vecteurs dans le référentiel de la fusée
    state : état de la fusée dans les coordonnées du référentiel terrestre
    """
    assert vectors.shape[-1]==3, 'l algo attend une suite de vecteurs 3D'
    assert len(state) == 6, 'l algo attend le vecteur état de la fusée dans le référentiel terrestre'
    
    alpha = np.radians(state[3])
    beta = np.radians(state[4])
    gamma = np.radians(state[5])
    
    R_x = np.array(( (1, 0, 0),
                    (0, np.cos(alpha), -np.sin(alpha)),
                    (0, np.sin(alpha),  np.cos(alpha)) ))
    
    R_y = np.array(((np.cos(beta), 0, np.sin(beta)),
                    (0, 1, 0),
                    (-np.sin(beta), 0,  np.cos(beta)) ))
    
    R_z = np.array(((np.cos(gamma), - np.sin(gamma), 0),
                    (np.sin(gamma), np.cos(gamma), 0),
                    (0, 0, 1) ))
    
    out=[]
    for vect in vectors:
        out.append(np.array(np.dot(R_z, np.dot(R_y, np.dot(R_x, vect)))))
    return np.array(out)

In [4]:
np.diag((2,6,3))

array([[2, 0, 0],
       [0, 6, 0],
       [0, 0, 3]])

In [7]:
np.radians(90)/np.pi

0.5

In [13]:
np.array(((0),(0),(1)))

array([0, 0, 1])

In [34]:
theta = np.pi/2
pelo =np.array(( 
                    (0, np.cos(theta), -np.sin(theta)),
                    (0, np.sin(theta),  np.cos(theta)) ))

In [39]:
pelo

array([[ 0.000000e+00,  6.123234e-17, -1.000000e+00],
       [ 0.000000e+00,  1.000000e+00,  6.123234e-17]])

In [50]:
coord_local2global(pelo, np.array([25, 69, 89, 5, 6, 9]))

array([[-0.11648284,  0.06979308, -0.99073744],
       [-0.14684109,  0.98535505,  0.08667829]])

In [43]:
len(np.array([25, 69, 89, 5, 6, 9]))

6